# Capstone Project

# Exploring Neighbourhoods in Toronto 
###### by Siddharaj Maramwar



Lets Install and import Required libraries

In [1]:
pip install BeautifulSoup4

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install lxml

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install geocoder

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install geopy

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install folium

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 94 kB 6.1 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
from bs4 import BeautifulSoup  # Web Crawler
import requests # used to make requests from web
import pandas as pd
import lxml
import geocoder
import os
import folium # map rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline
import lxml

In [8]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text

In [9]:
soup = BeautifulSoup(source, 'xml')
table=soup.find('table')

In [10]:
#our dataframe will have three columns "PostalCode, Borough, and Neighborhood"
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)
df

,Postalcode,Borough,Neighborhood


In [11]:
for tr in table.find_all('tr'):
    row_data=[]
    for td_cell in tr.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [12]:
df.head()

,Postalcode,Borough,Neighborhood


#### Cleaning of our data frame

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
And drop the rows where borough itself is Not assigned.

In [13]:
# Simply get the rows which has Borough
df=df[df['Borough']!='Not assigned']

Check if a value exists in a Borough column

In [14]:
exists = 'Not assigned' in df['Borough']
exists

False

Hence there in no cell in column 'Borough' having value => Not Assigned.

Now if a cell has a Not assigned in neighborhood column then use value of 'Borough'.

In [15]:
df[df['Neighborhood']=='Not assigned']=df['Borough'].reset_index()
df.head()

,Postalcode,Borough,Neighborhood


In [16]:
temp_df=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [17]:
df_merge = pd.merge(df, temp_df, on='Postalcode')
df_merge.drop(['Neighborhood'],axis=1,inplace=True)
df_merge.drop_duplicates(inplace=True)

,Borough,Postalcode,Neighborhood_joined


In [18]:
df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)
df_merge

,Borough,Postalcode,Neighborhood


In [19]:
df_merge.shape

(0, 3)

#### Part 2

Now we have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

I was not able to get the geographical coordinates of the neighborhoods using the Geocoder package, so I used
csv file.


link -> http://cocl.us/Geospatial_data

In [20]:
cord_df=pd.read_csv('http://cocl.us/Geospatial_data')

HTTPError: HTTP Error 308: Permanent Redirect

In [ ]:
cord_df.head()

In [ ]:
cord_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
cord_merged = pd.merge(cord_df, df_merge, on='Postalcode')

In [ ]:
geo_data=cord_merged[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]

In [ ]:
geo_data

We will work with boroughs that contain the word Toronto

In [ ]:
toronto_data=geo_data[geo_data['Borough'].str.contains("Toronto")]
toronto_data = toronto_data.drop(['Postalcode'], axis = 1)


In [ ]:
toronto_data

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [ ]:
CLIENT_ID = 'C2BMYANVYZY4N2F4KBR2BMBKBMS20RJ0OVOALAK44XFUWVAU' # your Foursquare ID
CLIENT_SECRET = 'RZ0OE3JMBPFPR5WHDRBDNGHY5H1DRQJPHJ35KY2HQOVABGMB' # your Foursquare Secret
ACCESS_TOKEN = 'OBIUEPVCLFO0XHE1VJYCSDS1JX5D1KUARQV2ZQTEZESR05FG' # your FourSquare Access Token
VERSION = '20210602'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
def geo_location(address):
    # get geo location of address
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    return latitude,longitude

#### We will define a function for getting the venues

In [ ]:
def get_venues(lat,lng):
    
    #set variables
    radius=1000
    LIMIT=100
   
    
    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
    
    # get all the data
    results = requests.get(url).json()
    venue_data=results["response"]['groups'][0]['items']
    venue_details=[]
    for row in venue_data:
        try:
            venue_id=row['venue']['id']
            venue_name=row['venue']['name']
            venue_category=row['venue']['categories'][0]['name']
            venue_details.append([venue_id,venue_name,venue_category])
        except KeyError:
            pass
        
    column_names=['ID','Name','Category']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df

##### Define function to retieve ratings and likes of the venue

In [ ]:
def get_venue_details(venue_id):
        
    
    
    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            venue_id,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
    
    # get all the data
    results = requests.get(url).json()
    venue_data=results['response']['venue']
    venue_details=[]
    try:
        venue_id=venue_data['id']
        venue_name=venue_data['name']
        venue_likes=venue_data['likes']['count']
        venue_rating=venue_data['rating']
        venue_tips=venue_data['tips']['count']
        venue_details.append([venue_id,venue_name,venue_likes,venue_rating,venue_tips])
    except KeyError:
        pass
        
    column_names=['ID','Name','Likes','Rating','Tips']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df

In [ ]:
plt.figure(figsize=(9,5), dpi = 100)
# title
plt.title('Number of Neighborhood for each Borough in Toronto')
#On x-axis
plt.xlabel('Borough', fontsize = 15)
#On y-axis
plt.ylabel('No.of Neighborhood', fontsize=15)
#giving a bar plot
toronto_data.groupby('Borough')['Neighborhood'].count().plot(kind='bar')
#legend
plt.legend()
#displays the plot
plt.show()

In [ ]:
# prepare neighborhood list that contains indian resturants
column_names=['Borough', 'Neighborhood', 'ID','Name']
indian_rest_to=pd.DataFrame(columns=column_names)
count=1
for row in toronto_data.values.tolist():
    Borough, Neighborhood, Latitude, Longitude=row
    venues = get_venues(Latitude,Longitude)
    indian_resturants=venues[venues['Category']=='Indian Restaurant']   
    print('(',count,'/',len(toronto_data),')','Indian Resturants in '+Neighborhood+', '+Borough+':'+str(len(indian_resturants)))
    for resturant_detail in indian_resturants.values.tolist():
        id, name , category=resturant_detail
        indian_rest_to = indian_rest_to.append({'Borough': Borough,
                                                'Neighborhood': Neighborhood, 
                                                'ID': id,
                                                'Name' : name
                                               }, ignore_index=True)
    count+=1

Now we got all the indian restaurants in the Toronto City

In [ ]:
indian_rest_to.head()

In [ ]:
indian_rest_to.shape

We got 27 Indian Resturants across Toronto City

In [ ]:
plt.figure(figsize=(9,5), dpi = 100)
# title
plt.title('Number of Indian Resturants for each Borough in Toronto City')
#On x-axis
plt.xlabel('Borough', fontsize = 15)
#On y-axis
plt.ylabel('No.of Indian Resturants', fontsize=15)
#giving a bar plot
indian_rest_to.groupby('Borough')['ID'].count().plot(kind='bar')
#legend
plt.legend()
#displays the plot
plt.show()

We see that East Toronto has the largest number of indian resturants

In [ ]:
plt.figure(figsize=(9,5), dpi = 100)
# title
plt.title('Number of Indian Resturants for each Neighborhood in Toronto City')
#On x-axis
plt.xlabel('Neighborhood', fontsize = 15)
#On y-axis
plt.ylabel('No.of Indian Resturants', fontsize=15)
#giving a bar plot
indian_rest_to.groupby('Neighborhood')['ID'].count().nlargest(5).plot(kind='bar')
#legend
plt.legend()
#displays the plot
plt.show()

From Above we can observe the Neighborhood named "India Bazar" and "The Beaches West" has most number of 
Indian Restaurants.
If you look in the Postal Code dataframe (df)  you will see both Neighborhood has same postal code i.e,  M4L

From above we can tell the postal code with M4L has the maximum number of Indian Restaurants.

Now we will get the ranking of each resturant for further analysis.

In [ ]:
#prepare neighborhood list that contains indian resturants
column_names=['Borough', 'Neighborhood', 'ID','Name','Likes','Rating','Tips']
indian_rest_stats_to=pd.DataFrame(columns=column_names)
count=1


for row in indian_rest_to.values.tolist():
    Borough,Neighborhood,ID,Name=row
    try:
        venue_details=get_venue_details(ID)
        print(venue_details)
        id,name,likes,rating,tips=venue_details.values.tolist()[0]
    except IndexError:
        print('No data available for id=',ID)
        # we will assign 0 value for these resturants as they may have been 
        #recently opened or details does not exist in FourSquare Database
        id,name,likes,rating,tips=[0]*5
    print('(',count,'/',len(indian_rest_to),')','processed')
    indian_rest_stats_to = indian_rest_stats_to.append({'Borough': Borough,
                                                'Neighborhood': Neighborhood, 
                                                'ID': id,
                                                'Name' : name,
                                                'Likes' : likes,
                                                'Rating' : rating,
                                                'Tips' : tips
                                               }, ignore_index=True)
    count+=1

In [ ]:
indian_rest_stats_to.head()

Hurray!!
Now I fetched the Likes , Ratings, and Tips(user reviews) of the Indian restaurants in the city.

In [ ]:
indian_rest_stats_to.shape

In [ ]:
indian_rest_to.shape

From above we can observe we have fetched all data required for all the Indian restaurants in the city

Lets save this in CSV form

In [ ]:
indian_rest_stats_to.to_csv('indian_rest_stats_to.csv', index=False)

Let's verify the saves csv file

In [ ]:
indian_rest_stats_to_csv=pd.read_csv('indian_rest_stats_to.csv')

In [ ]:
indian_rest_stats_to_csv

In [ ]:
indian_rest_stats_to_csv.shape

In [ ]:
indian_rest_stats_to.info()

We see that values like Likes, Tips are string values. We need to convert them into float values for further analysis.

In [ ]:
indian_rest_stats_to[['Likes','Tips']]=indian_rest_stats_to[['Likes','Tips']].astype('float64')

In [ ]:
indian_rest_stats_to.info()

Now Likes, Rating, Tips are converted to float and can be used for further analysis.

Lets find out average rating of resturants in the particular Neighborhood

In [ ]:
to_neighborhood_stats=indian_rest_stats_to.groupby('Neighborhood',as_index=False).mean()[['Neighborhood','Rating']]
to_neighborhood_stats.columns=['Neighborhood','Average Rating']

Let's Sort them according to the Ratings they achieved

In [ ]:
to_neighborhood_stats.sort_values(['Average Rating'],ascending=False).head(10)

We see Neighborhood 'Christie' has Maximum Rating of 8.45

Similarly lets do it for Borough

In [ ]:
to_borough_stats=indian_rest_stats_to.groupby('Borough',as_index=False).mean()[['Borough','Rating']]
to_borough_stats.columns=['Borough','Average Rating']

In [ ]:
#Sort them
to_borough_stats.sort_values(['Average Rating'],ascending=False).head()


Borough Downtown Toronto has the Max Ratings with 7.92

Lets visualize it.

In [ ]:
plt.figure(figsize=(10,7), dpi = 100)
# title
plt.title('Average rating of Indian Resturants for each Borough')
#On x-axis
plt.xlabel('Borough', fontsize = 15)
#On y-axis
plt.ylabel('Average Rating', fontsize=15)
#giving a bar plot
indian_rest_stats_to.groupby('Borough').mean()['Rating'].plot(kind='bar')
#legend
plt.legend()
#displays the plot
plt.show()

Now we will consider all the neighborhoods with average rating greater or equal 6.0 to visualize on map

In [ ]:
to_neighborhood_stats=to_neighborhood_stats[to_neighborhood_stats['Average Rating']>=6.0]

In [ ]:

to_neighborhood_stats.sort_values(['Average Rating'],ascending=False).head()

Now we will merge the toronto dataset to get Latitude and Longitude

In [ ]:
to_neighborhood_stats=pd.merge(to_neighborhood_stats,toronto_data, on='Neighborhood')

In [ ]:
to_neighborhood_stats=to_neighborhood_stats[['Borough','Neighborhood','Latitude','Longitude','Average Rating']]

In [ ]:
to_neighborhood_stats

Now above we have the restaurants which has ratings grater or equal to 6.0 and their Geo-Co-ordinates

Lets Visualise it!

In [ ]:
to_map = folium.Map(location=geo_location('Toronto'), zoom_start=12)

In [ ]:
to_map

In [ ]:
# instantiate a feature group for the incidents in the dataframe
incidents = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for lat, lng, in to_neighborhood_stats[['Latitude','Longitude']].values:
    incidents.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=10, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

In [ ]:
to_neighborhood_stats['Label']=to_neighborhood_stats['Neighborhood']+', '+to_neighborhood_stats['Borough']+'('+to_neighborhood_stats['Average Rating'].map(str)+')'

In [ ]:
# add pop-up text to each marker on the map
for lat, lng, label in to_neighborhood_stats[['Latitude','Longitude','Label']].values:
    folium.Marker([lat, lng], popup=label).add_to(to_map)        
# add incidents to map
to_map.add_child(incidents)

In [ ]:
to_map = folium.Map(location=geo_location('Toronto'), zoom_start=12)
to_geo = r'https://github.com/jasonicarter/toronto-geojson/blob/master/toronto_crs84.geojson'

to_map.choropleth(
    geo_data=to_geo,
    data=to_borough_stats,
    columns=['Borough', 'Average Rating'],
    key_on='feature.properties.boro_name',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Average Rating'
)

# display map
# as this is huge map data , we will save it to a file
ny_map.save('borough_rating.html')

In [ ]:
to_neighborhood_stats.sort_values(['Average Rating'],ascending=False).head(10)

## Conclusion

#### Below are the Best Neighborhoods to open an Indian Restaurant

Christie,High Park,The Junction South, The Annex, North Midtown, Yorkville, Church and Wellesley, Queen's Park, Ontario Provincial Government, St. James Town, Cabbagetown

### Limitations 

1. The Results are highly dependent on the ratings of the Restaurants.
2. The Rating Accuracy is highly dependent on Foursquare API.